# Importing libraries and Reading Data

In [ ]:
pip install pyarrow


In [ ]:
import pandas as pd
import numpy as np


import time
from sklearn.ensemble import GradientBoostingRegressor

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import missingno as msno
from tabulate import tabulate

# Check if the data is normally distributed
from scipy.stats import shapiro, boxcox
from sklearn.preprocessing import PowerTransformer
import statsmodels.api as sm

# Preprocessing
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, MultiLabelBinarizer, LabelEncoder
import re
import uuid



# bronze layer

In [ ]:
df = pd.read_csv('Property_uncleaned.csv')

In [ ]:
df.shape

(4525, 11)

####  11 columns , but   there some numerical columns as obj

In [ ]:
df.head().style.background_gradient('Blues')

,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
0,2 BHK Apartment for Sale in Dindoli Surat,Carpet Area,644 sqft,New Property,Poss. by Oct '24,5 out of 10,Unfurnished,West,"Luxury project with basement parking, Solar rooftop, Gym, Build with advanced machinery and standard materials that last long. Best project for under construction scheme for a happy family life.Read more","₹2,891 per sqft",₹33.8 Lac
1,2 BHK Apartment for Sale in Althan Surat,Super Area,1278 sqft,New Property,Poss. by Jan '26,6 out of 14,Unfurnished,South -West,2 And 3 BHK Luxurious Flat for Sell In New Althan Area With All Aminities .Read more,"₹3,551 per sqft",₹45.4 Lac
2,2 BHK Apartment for Sale in Pal Gam Surat,Super Area,1173 sqft,Resale,Ready to Move,5 out of 13,Semi-Furnished,East,"This affordable 2 BHK flat is situated along a 200foot road, ensuring easy access and connectivity. Enjoy the convenience of a prime location at a budgetfriendly price. Additionally, it is ready to move in, offering immediate occupancy for your convenience. With a towering 19floor height, it provides impressive views and a sense of elevation.Read more","₹3,800 per sqft",₹44.6 Lac
3,2 BHK Apartment for Sale in Jahangirabad Surat,Carpet Area,700 sqft,New Property,Ready to Move,6 out of 14,Unfurnished,East,2 BHK Flat For sell IN Jahangirabad Prime Location,"₹3,966 per sqft",₹47 Lac
4,"2 BHK Apartment for Sale in Orchid Fantasia, Palanpur Surat",Super Area,1250 sqft,Orchid Fantasia,New Property,Unfurnished,2,2,"Multistorey Apartment for Sale in Palanpur, Surat. Covered area is 1250.0 Sq-ft. This property belongs to ""Orchid Fantasia"" .Read more","₹3,600 per sqft",₹45 Lac


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4525 entries, 0 to 4524
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   property_name   4525 non-null   object
 1   areaWithType    4525 non-null   object
 2   square_feet     4525 non-null   object
 3   transaction     4421 non-null   object
 4   status          4524 non-null   object
 5   floor           4480 non-null   object
 6   furnishing      4185 non-null   object
 7   facing          3936 non-null   object
 8   description     3154 non-null   object
 9   price_per_sqft  4157 non-null   object
 10  price           4525 non-null   object
dtypes: object(11)
memory usage: 389.0+ KB


In [ ]:
df.isnull().sum()


,0
property_name,0
areaWithType,0
square_feet,0
transaction,104
status,1
floor,45
furnishing,340
facing,589
description,1371
price_per_sqft,368


In [ ]:
df.duplicated().sum()

np.int64(109)

In [ ]:
# df.drop_duplicates(inplace=True)


we have 109 duplicate rows and dropped them

In [ ]:
duplicate_rows = df[df.duplicated()]
duplicate_rows

,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
1511,2 BHK Apartment for Sale in NavYug College Surat,Carpet Area,1650 sqft,Resale,Ready to Move,7 out of 10,Unfurnished,East,"Presenting an exceptional 2 BHK flat for sale,...","₹3,125 per sqft",₹50 Lac
1512,2 BHK Apartment for Sale in Adajan Surat,Super Area,847 sqft,Resale,Ready to Move,1 out of 13,Furnished,East,"2 BHK, Multistorey Apartment is available for ...","₹5,903 per sqft",₹50 Lac
1535,2 BHK Apartment for Sale in Palanpur Surat,Super Area,1326 sqft,Resale,Ready to Move,9 out of 14,Unfurnished,East,2 BHK 3 BHK apartments at palanpur. with all...,"₹3,771 per sqft",₹50 Lac
1566,2 BHK Builder Floor for Sale in Nana Varachha ...,Super Area,1267 sqft,Resale,Ready to Move,2 out of 4,Furnished,North - West,"2 BHK, Builder Floor Apartment is available fo...","₹3,946 per sqft",₹50 Lac
1571,2 BHK Apartment for Sale in Kamrej Road Surat,Carpet Area,783 sqft,Resale,Ready to Move,5 out of 5,Unfurnished,North - East,Multistorey apartment is available for sale. I...,NaN,₹50 Lac
...,...,...,...,...,...,...,...,...,...,...,...
3870,4 BHK Builder Floor for Sale in New citylight ...,Carpet Area,2800 sqft,Furnished,2 out of 12,New Property,North - West,Main Road,"Flat fully furnished, with 4 bedrooms, all lux...",NaN,Call for Price
3903,5 BHK House for Sale in Jahangir Pura Surat,Carpet Area,3400 sqft,Semi-Furnished,Ready to Move,Resale,North - East,Main Road,new property,"₹15,152 per sqft",₹1.50 Cr
4152,3 BHK Apartment for Sale in Vesu Surat,Super Area,2500 sqft,New Property,Poss. by Mar '25,7 out of 13,Unfurnished,South -West,3 BHK Flat For Sell In Vesu Prime Area With Al...,"₹6,000 per sqft",₹1.50 Cr
4273,"3 BHK Apartment for Sale in Sky Superia, VIP R...",Super Area,2400 sqft,New Property,Ready to Move,10 out of 19,Unfurnished,East,There are apartments for sale in Sky Superia. ...,"₹6,250 per sqft",₹1.50 Cr


duplicates seems to be not duplicates just different appartments

In [ ]:
df.nunique()

,0
property_name,1992
areaWithType,6
square_feet,1399
transaction,38
status,138
floor,222
furnishing,78
facing,176
description,2588
price_per_sqft,2134


* Categorical Columns:

areaWithType (6), transaction (38), status (138), floor (222), furnishing (78), facing (176)

* Missing Data Present In:

transaction, status, floor, furnishing, facing, description, price_per_sqft

In [ ]:
df.dtypes

,0
property_name,object
areaWithType,object
square_feet,object
transaction,object
status,object
floor,object
furnishing,object
facing,object
description,object
price_per_sqft,object


In [ ]:
df.isnull().sum()

,0
property_name,0
areaWithType,0
square_feet,0
transaction,104
status,1
floor,45
furnishing,340
facing,589
description,1371
price_per_sqft,368


Will extract numbers from price and price_per_sqft

Normalize square_feet to int

Split floor into current floor / total floors

Flag rows with missing essential features for later review

first will save bronze layer


In [ ]:
df.to_parquet("bronze_layer_real_estate.parquet", index=False)


#silver layer

## areaWithType column

In [ ]:
unique_area_types = df['areaWithType'].unique()
print(unique_area_types)

['Carpet Area' 'Super Area' 'Plot Area' 'Built Area' 'Transaction'
 'Status']


In [ ]:
area_type_counts = df['areaWithType'].value_counts()

print("\nCount of each unique value in ")
print(area_type_counts)



Count of each unique value in 
areaWithType
Super Area     2599
Carpet Area    1671
Plot Area       214
Built Area       35
Status            4
Transaction       2
Name: count, dtype: int64


will inspect Status and Transaction as they seems outliers

In [ ]:
df[df['areaWithType'] == 'Transaction']

,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
324,1 BHK House for Sale in Jolva Surat,Transaction,Resale,NaN,2,1,NaN,NaN,NaN,NaN,₹12 Lac
950,House for Sale in Kamrej Surat,Transaction,Resale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,₹24 Lac


will investigate if there's any specific meaning to the rows with "Transaction" in areaWithType and handle them accordingly

In [ ]:
df[df['areaWithType'] == 'Status']

,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
368,Office Space for Sale in Lal Gate Surat,Status,Ready to Move,1,3 out of 5,Resale,NaN,NaN,NaN,NaN,₹30 Lac
471,1 BHK Apartment for Sale in Morabhagal Surat,Status,Ready to Move,Unfurnished,13 out of 14,Resale,1,NaN,NaN,NaN,₹15 Lac
631,2 BHK Apartment for Sale in kanakpur kansad Surat,Status,Ready to Move,Unfurnished,Ground out of 10,Resale,East,Garden/Park,Urgently SellingGood Condition Ground Floor Ga...,NaN,₹22 Lac
846,2 BHK Apartment for Sale in Majura Gate Surat,Status,Ready to Move,Furnished,4 out of 5,Resale,2,2,This spacious Multistorey Apartment is locate...,NaN,₹30 Lac


In [ ]:
valid_values = ['Carpet Area', 'Super Area']
df['areaWithType'] = df['areaWithType'].apply(lambda x: x if x in valid_values else np.nan)

In [ ]:
print("\nUnique values after replacement:")
print(df['areaWithType'].unique())


Unique values after replacement:
['Carpet Area' 'Super Area' nan]


## price

In [ ]:
df[["price"]]


,price
0,₹33.8 Lac
1,₹45.4 Lac
2,₹44.6 Lac
3,₹47 Lac
4,₹45 Lac
...,...
4520,Call for Price
4521,Call for Price
4522,Call for Price
4523,Call for Price


In [ ]:
call_for_price_rows = df[df['price'] == 'Call for Price']

# Show the result
call_for_price_rows

,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
1014,2 BHK Apartment for Sale in Chauta Bazar Surat,Carpet Area,1100 sqft,Resale,Ready to Move,5 out of 5,Semi-Furnished,North,Welcome to an exceptional opportunity in Chaut...,NaN,Call for Price
1015,1 BHK Apartment for Sale in Vesu Surat,Carpet Area,366 sqft,Resale,Ready to Move,2 out of 8,Unfurnished,Co-operative Society,NaN,NaN,Call for Price
1016,2 BHK Apartment for Sale in Nanpura Surat,Carpet Area,820 sqft,Resale,Ready to Move,4 out of 9,Furnished,West,"2 front side balcony west side view,main road ...",NaN,Call for Price
1017,2 BHK Apartment for Sale in Amroli Surat,Super Area,760 sqft,Resale,Ready to Move,1 out of 1,Unfurnished,East,Iron Grills Safety Provided to all windows and...,NaN,Call for Price
1018,2 BHK Apartment for Sale in Gopi Pura Surat,Carpet Area,1000 sqft,Resale,Ready to Move,2 out of 3,Semi-Furnished,1,NaN,NaN,Call for Price
...,...,...,...,...,...,...,...,...,...,...,...
4520,6 BHK Apartment for Sale in Millionaires Lifes...,Carpet Area,2000 sqft,New Property,Poss. by Dec '26,5 out of 12,Unfurnished,South - East,"Check out Millionaires Lifestyle in Vesu, one ...",NaN,Call for Price
4521,"4 BHK Apartment for Sale in Savan Superia, Alt...",Super Area,3600 sqft,New Property,Poss. by Dec '25,5 out of 16,Unfurnished,South - East,Superia is a premium residential project launc...,NaN,Call for Price
4522,5 BHK Apartment for Sale in Roongta Green Vall...,Carpet Area,2250 sqft,New Property,Poss. by Dec '25,7 out of 13,Unfurnished,North - East,"When it comes to beautiful homes, nothing beat...",NaN,Call for Price
4523,"6 BHK Apartment for Sale in Cellestial Dreams,...",Carpet Area,3450 sqft,New Property,Ready to Move,7 out of 18,Unfurnished,North - West,"DRB Ravani Cellestial Dreams in Vesu, Surat is...",NaN,Call for Price


In [ ]:
# some people didint give the price neither price_per_sqft will be 0

In [ ]:
df["price"].unique()

array(['₹33.8 Lac ', '₹45.4 Lac ', '₹44.6 Lac ', '₹47 Lac ', '₹45 Lac ',
       '₹43.2 Lac ', '₹42.1 Lac ', '₹44.1 Lac ', '₹44.3 Lac ', '₹40 Lac ',
       '₹44 Lac ', '₹12.5 Lac ', '₹50 Lac ', '₹35.8 Lac ', '₹26.5 Lac ',
       '₹42 Lac ', '₹43 Lac ', '₹20 Lac ', '₹37 Lac ', '₹29 Lac ',
       '₹38 Lac ', '₹13 Lac ', '₹46.5 Lac ', '₹41.6 Lac ', '₹33.5 Lac ',
       '₹42.5 Lac ', '₹48 Lac ', '₹32 Lac ', '₹35 Lac ', '₹46.2 Lac ',
       '₹38.5 Lac ', '₹23 Lac ', '₹24 Lac ', '₹28 Lac ', '₹35.7 Lac ',
       '₹36.5 Lac ', '₹36 Lac ', '₹43.6 Lac ', '₹25 Lac ', '₹41 Lac ',
       '₹28.5 Lac ', '₹9 Lac ', '₹30 Lac ', '₹39.5 Lac ', '₹21.5 Lac ',
       '₹10.5 Lac ', '₹18 Lac ', '₹10 Lac ', '₹23.5 Lac ', '₹16.3 Lac ',
       '₹13.5 Lac ', '₹49 Lac ', '₹21 Lac ', '₹39 Lac ', '₹16 Lac ',
       '₹27 Lac ', '₹15 Lac ', '₹39.9 Lac ', '₹7.5 Lac ', '₹12.8 Lac ',
       '₹9.5 Lac ', '₹31.5 Lac ', '₹46 Lac ', '₹6 Lac ', '₹34 Lac ',
       '₹22 Lac ', '₹19 Lac ', '₹8.5 Lac ', '₹11 Lac ', '₹16.5 Lac ',
 

In [ ]:
def clean_price(price):
    try:
        price = price.replace('₹', '').strip()

        if 'Crore' in price:
            price = price.replace(' Crore', '').strip()
            return float(price) * 10**7

        elif 'Lac' in price:
            price = price.replace(' Lac', '').strip()
            return float(price) * 10**5

        else:
            return float(price)

    except Exception:
        return 0.0

df['price'] = df['price'].apply(clean_price)

# Show the cleaned data
print(df)


                                          property_name areaWithType  \
0             2 BHK Apartment for Sale in Dindoli Surat  Carpet Area   
1              2 BHK Apartment for Sale in Althan Surat   Super Area   
2             2 BHK Apartment for Sale in Pal Gam Surat   Super Area   
3        2 BHK Apartment for Sale in Jahangirabad Surat  Carpet Area   
4     2 BHK Apartment for Sale in Orchid Fantasia, P...   Super Area   
...                                                 ...          ...   
4520  6 BHK Apartment for Sale in Millionaires Lifes...  Carpet Area   
4521  4 BHK Apartment for Sale in Savan Superia, Alt...   Super Area   
4522  5 BHK Apartment for Sale in Roongta Green Vall...  Carpet Area   
4523  6 BHK Apartment for Sale in Cellestial Dreams,...  Carpet Area   
4524  4 BHK Apartment for Sale in Roongta Green Vall...   Super Area   

     square_feet      transaction            status        floor  \
0       644 sqft     New Property  Poss. by Oct '24  5 out of 10   

In [ ]:
fig = px.histogram(df, x="price", nbins=50,
                   title="Distribution of Property Prices",
                   labels={"price": "Price"})
fig.update_layout(bargap=0.1)
fig.show()

the most frequent price is 250 - 240

## square_feet

In [ ]:
df[["square_feet"]].head()

,square_feet
0,644 sqft
1,1278 sqft
2,1173 sqft
3,700 sqft
4,1250 sqft


In [ ]:
df['square_feet'] = df['square_feet'].str.replace(r'[^\d.]', '', regex=True)

df['square_feet'] = pd.to_numeric(df['square_feet'], errors='coerce')

df['square_feet'] = df['square_feet'].fillna(0).astype('Int64')

df[["square_feet"]].head()


,square_feet
0,644
1,1278
2,1173
3,700
4,1250


In [ ]:
df['square_feet'].unique()

<IntegerArray>
[ 644, 1278, 1173,  700, 1250, 1265, 1404, 1180,  720,  748,
 ...
 2824, 4730, 2881, 3945, 2282, 4580, 2970, 2545, 5175, 2620]
Length: 1346, dtype: Int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4525 entries, 0 to 4524
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   property_name   4525 non-null   object 
 1   areaWithType    4270 non-null   object 
 2   square_feet     4525 non-null   Int64  
 3   transaction     4421 non-null   object 
 4   status          4524 non-null   object 
 5   floor           4480 non-null   object 
 6   furnishing      4185 non-null   object 
 7   facing          3936 non-null   object 
 8   description     3154 non-null   object 
 9   price_per_sqft  4157 non-null   object 
 10  price           4525 non-null   float64
dtypes: Int64(1), float64(1), object(9)
memory usage: 393.4+ KB


## transaction

In [ ]:
print(df['transaction'].unique())


['New Property' 'Resale' 'Orchid Fantasia' 'Unfurnished' 'Orchid Gardenia'
 'Congo' 'Furnished' nan 'Semi-Furnished' 'Main Road, Garden/Park' '1' '3'
 '4' 'Yes' 'No' 'Mahavir Nagar' '3 Covered' 'Shagun Residency' '2'
 'Main Road' '10 m' 'Other' '1 Covered' 'Soham Elegance' 'Garden/Park'
 'Aakash Echo Point' 'Sundaram Residency' '5' 'Jolly Residency'
 'Mangalam Heights' 'Green City' 'Pool, Garden/Park, Main Road' '6'
 'Co-operative Society' 'Vatsalya Bungalows' '20 Covered,'
 'Raghuvir Silverstone' '8 m' 'Casa Rivera']


In [ ]:
transaction_counts = df['transaction'].value_counts()

print("\nCount of each unique value in ")
print(transaction_counts)


Count of each unique value in 
transaction
Resale                          2197
New Property                    1486
Unfurnished                      404
Semi-Furnished                    91
Furnished                         69
2                                 24
3                                 22
No                                20
Congo                             19
1                                 16
Yes                               15
5                                 11
4                                  7
Main Road                          6
Soham Elegance                     3
Orchid Gardenia                    3
6                                  3
Orchid Fantasia                    2
Jolly Residency                    2
Raghuvir Silverstone               2
Casa Rivera                        2
Main Road, Garden/Park             1
1 Covered                          1
Other                              1
10 m                               1
Shagun Residency               

In [ ]:
# Filter rows where 'transaction' is in the specified list and 'furnishing' is not one of the three values
filtered_rows = df[(df['transaction'].isin(['Unfurnished', 'Furnished', 'Semi-Furnished'])) & (~df['furnishing'].isin(['Unfurnished', 'Furnished', 'Semi-Furnished']))]

print(filtered_rows)


                                          property_name areaWithType  \
5     2 BHK Apartment for Sale in Anand Aspire, Jaha...   Super Area   
6                 3 BHK Villa for Sale in Dindoli Surat  Carpet Area   
22    2 BHK Apartment for Sale in Palan Pur Patiya S...   Super Area   
26           2 BHK Apartment for Sale in Palanpur Surat  Carpet Area   
28          3 BHK Villa for Sale in Masma Village Surat  Carpet Area   
...                                                 ...          ...   
4377  4 BHK Apartment for Sale in Bblossom The Presi...   Super Area   
4384  4 BHK Apartment for Sale in Bblossom The Presi...   Super Area   
4390  4 BHK Apartment for Sale in Bblossom The Presi...   Super Area   
4396  4 BHK Apartment for Sale in Bblossom The Presi...   Super Area   
4402  4 BHK Apartment for Sale in Bblossom The Presi...   Super Area   

      square_feet  transaction            status         floor  \
5            1265  Unfurnished  Poss. by Dec '25  New Property   
6  

In [ ]:
# Filter rows where 'transaction' is in the specified list and 'furnishing' is not one of the three values
filtered_rows = df[(df['transaction'].isin(['Unfurnished', 'Furnished', 'Semi-Furnished'])) & (~df['furnishing'].isin(['Unfurnished', 'Furnished', 'Semi-Furnished']))]

# Update the 'furnishing' column with the corresponding 'transaction' value for the filtered rows
df.loc[filtered_rows.index, 'furnishing'] = df.loc[filtered_rows.index, 'transaction']

# Count how many rows in 'furnishing' were filled with values from 'transaction'
filled_count = df.loc[filtered_rows.index, 'furnishing'].notna().sum()

# Print the count
print(filled_count)



564


In [ ]:
filtered_rows = df[(df['transaction'].isin(['Unfurnished', 'Furnished', 'Semi-Furnished'])) & (df['furnishing'] == 'nan')]

print(filtered_rows)

Empty DataFrame
Columns: [property_name, areaWithType, square_feet, transaction, status, floor, furnishing, facing, description, price_per_sqft, price]
Index: []


some wrong data entry , will keep only 'New Property' 'Resale'

In [ ]:
# Filter rows where any column (except 'transaction') contains 'New Property' or 'Resale'
filtered_rows = df[df.drop(columns=['transaction']).apply(lambda row: row.astype(str).str.contains('New Property|Resale', case=False).any(), axis=1)]

# Display the filtered rows
print(filtered_rows)

# Count how many rows match the condition
count = filtered_rows.shape[0]
print(f"Count of rows having 'New Property' or 'Resale' in any column (excluding 'transaction'): {count}")


                                          property_name areaWithType  \
4     2 BHK Apartment for Sale in Orchid Fantasia, P...   Super Area   
5     2 BHK Apartment for Sale in Anand Aspire, Jaha...   Super Area   
6                 3 BHK Villa for Sale in Dindoli Surat  Carpet Area   
8     2 BHK Apartment for Sale in Orchid Gardenia, P...   Super Area   
22    2 BHK Apartment for Sale in Palan Pur Patiya S...   Super Area   
...                                                 ...          ...   
4377  4 BHK Apartment for Sale in Bblossom The Presi...   Super Area   
4384  4 BHK Apartment for Sale in Bblossom The Presi...   Super Area   
4390  4 BHK Apartment for Sale in Bblossom The Presi...   Super Area   
4396  4 BHK Apartment for Sale in Bblossom The Presi...   Super Area   
4402  4 BHK Apartment for Sale in Bblossom The Presi...   Super Area   

      square_feet      transaction            status         floor  \
4            1250  Orchid Fantasia      New Property   Unfurnishe

In [ ]:
# Calculate percentage of values in each column that contain 'New Property' or 'Resale'
percentage = df.apply(lambda col: col.astype(str).str.contains('New Property|Resale', case=False).mean() * 100)

# Count how many rows in each column contain 'New Property' or 'Resale'
count = df.apply(lambda col: col.astype(str).str.contains('New Property|Resale', case=False).sum())

# Display the percentage and count for each column
print("Percentage of values containing 'New Property' or 'Resale':")
print(percentage)

print("\nCount of rows containing 'New Property' or 'Resale':")
print(count)


Percentage of values containing 'New Property' or 'Resale':
property_name      0.000000
areaWithType       0.000000
square_feet        0.000000
transaction       81.392265
status             2.585635
floor             15.425414
furnishing         0.000000
facing             0.000000
description        0.839779
price_per_sqft     0.000000
price              0.000000
dtype: float64

Count of rows containing 'New Property' or 'Resale':
property_name        0
areaWithType         0
square_feet          0
transaction       3683
status             117
floor              698
furnishing           0
facing               0
description         38
price_per_sqft       0
price                0
dtype: int64


so will impute transaction with status , floor , description

In [ ]:
valid_values = ['New Property', 'Resale']
df['transaction'] = df['transaction'].apply(lambda x: x if x in valid_values else np.nan)

print("\nUnique values after replacement:")
print(df['transaction'].unique())



Unique values after replacement:
['New Property' 'Resale' nan]


In [ ]:
# Fill 'transaction' where 'floor' contains 'New Property' or 'Resale' and 'transaction' is NaN
df.loc[(df['floor'].str.contains('New Property|Resale', case=False, na=False)) & (df['transaction'].isna()), 'transaction'] = df['floor']

# Count how many rows are filled in the 'transaction' column
filled_count = df['transaction'].isna().sum()

# Display the number of rows filled
print(f"Number of rows filled in 'transaction': {filled_count}")


Number of rows filled in 'transaction': 144


In [ ]:
# Filter rows where 'description' contains 'New Property' or 'Resale'
filtered_rows = df[df['description'].str.contains('New Property|Resale', case=False, na=False)]

# Display the filtered rows
filtered_rows

# Count the number of rows
count = filtered_rows.shape[0]
print(f"Number of rows where 'description' is 'New Property' or 'Resale': {count}")


Number of rows where 'description' is 'New Property' or 'Resale': 38


In [ ]:
# Filter rows where 'description' contains 'New Property' or 'Resale'
filtered_rows = df[df['description'].str.contains('New Property|Resale', case=False, na=False)]


# Count the number of rows
count = filtered_rows.shape[0]
print(f"Number of rows where 'description' is 'New Property' or 'Resale': {count}")


Number of rows where 'description' is 'New Property' or 'Resale': 38


In [ ]:
df['transaction'].isna().sum()

np.int64(144)

In [ ]:
# Filter rows where 'transaction' is NaN
filtered_rows_na = df[df['transaction'].isna()]

# Display the filtered rows
filtered_rows_na


,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
4,"2 BHK Apartment for Sale in Orchid Fantasia, P...",Super Area,1250,NaN,New Property,Unfurnished,2,2,"Multistorey Apartment for Sale in Palanpur, Su...","₹3,600 per sqft",4500000.0
8,"2 BHK Apartment for Sale in Orchid Gardenia, P...",Super Area,1180,NaN,New Property,Unfurnished,2,2,"Multistorey Apartment for Sale in Palanpur, Su...","₹3,751 per sqft",4430000.0
90,Plot/Land for Sale in Bhestan Surat,NaN,1040,NaN,Resale,NaN,NaN,NaN,1040 Sq-ft Residential Plot is available for S...,"₹4,712 per sqft",4900000.0
124,Industrial Land for Sale in Surat City Surat,NaN,1318,NaN,Resale,NaN,NaN,NaN,1318 Sq-ft Industrial Land is available for Sa...,"₹1,669 per sqft",2200000.0
131,Industrial Land for Sale in Palsana Surat,NaN,1978,NaN,New Property,NaN,NaN,NaN,"1. Prime Location at NH 8 , Palsana.2. Easy Tr...","₹1,618 per sqft",3200000.0
...,...,...,...,...,...,...,...,...,...,...,...
3984,5 BHK Builder Floor for Sale in Raghuvir Silve...,Super Area,5700,NaN,New Property,Unfurnished,1 Open,5,"Builder Floor Apartment for Sale in Vesu, Sura...","₹6,500 per sqft",0.0
4012,4 BHK Apartment for Sale in Raghuvir Silversto...,Super Area,3636,NaN,New Property,Unfurnished,4,4,"Multistorey Apartment for Sale in Vesu, Surat....","₹6,500 per sqft",0.0
4091,Land for Sale in Nanpura Surat,NaN,1800,NaN,Resale,NaN,NaN,NaN,business location area,"₹22,222 per sqft",0.0
4149,"5 BHK Apartment for Sale in Casa Rivera, Adaja...",Super Area,5000,NaN,New Property,Unfurnished,5,4,"Multistorey Apartment for Sale in Adajan, Sura...","₹7,500 per sqft",0.0


## price_per_sqft

In [ ]:
print(df['price_per_sqft'].isna().sum())


368


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4525 entries, 0 to 4524
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   property_name   4525 non-null   object 
 1   areaWithType    4270 non-null   object 
 2   square_feet     4525 non-null   Int64  
 3   transaction     4381 non-null   object 
 4   status          4524 non-null   object 
 5   floor           4480 non-null   object 
 6   furnishing      4185 non-null   object 
 7   facing          3936 non-null   object 
 8   description     3154 non-null   object 
 9   price_per_sqft  4157 non-null   object 
 10  price           4525 non-null   float64
dtypes: Int64(1), float64(1), object(9)
memory usage: 393.4+ KB


### so this may affect golden layer

In [ ]:
df['price_per_sqft'] = df['price_per_sqft'].str.replace(r'[^\d.]', '', regex=True)

## status

In [ ]:
df.isna().sum()

,0
property_name,0
areaWithType,255
square_feet,0
transaction,144
status,1
floor,45
furnishing,340
facing,589
description,1371
price_per_sqft,368


In [ ]:
df['status'].unique()

array(["Poss. by Oct '24", "Poss. by Jan '26", 'Ready to Move',
       'New Property', "Poss. by Dec '25", "Poss. by Jun '24",
       "Poss. by Sep '25", "Poss. by Feb '26", '1 out of 7',
       "Poss. by Jul '24", "Poss. by Dec '26", "Poss. by Apr '25",
       "Poss. by Jan '25", 'Freehold', '21 X 50.2', "Poss. by Dec '24",
       "Poss. by Aug '25", "Poss. by Feb '25", '3 out of 10', 'Resale',
       'Co-operative Society', '120 X 105', '3 out of 7',
       "Poss. by May '26", '36 X 15', "Poss. by Aug '24", '2',
       '3 out of 3', '52.5 X 32.1', '3 out of 5', '5 out of 5',
       '12 X 35.92', '13 out of 14', '7.29 X 14', '4 out of 5',
       'Ground out of 1', '3 out of 4', 'Ground out of 10',
       'Ground out of 2', '20 X 40', '16.4 X 60', '11 out of 14',
       '9.42 X 126', '25 X 12', nan, '2 out of 4', '54 X 15', '21 X 57',
       'Power Of Attorney', "Poss. by Jul '25", "Poss. by May '25",
       "Poss. by Mar '26", "Poss. by Jun '26", "Poss. by Nov '24",
       '26.7 X 55.

In [ ]:
df[['status']].head()

,status
0,Poss. by Oct '24
1,Poss. by Jan '26
2,Ready to Move
3,Ready to Move
4,New Property


In [ ]:
nan_before = df['transaction'].isna().sum()

df.loc[(df['transaction'].isna()) & (df['status'] == 'New Property'), 'transaction'] = 'New Property'
df.loc[(df['transaction'].isna()) & (df['status'] == 'Resale'), 'transaction'] = 'Resale'

nan_after = df['transaction'].isna().sum()

filled_count = nan_before - nan_after

print(f"Filled {filled_count} transaction values.")


Filled 117 transaction values.


#### that sound good as imputed some nan values in transaction

In [ ]:
# Find rows where status matches pattern like "number out of number"
mask = df['status'].astype(str).str.match(r'^\d+\s*out\s*of\s*\d+$')

# Show those rows
matching_rows = df[mask]

matching_rows


,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
22,2 BHK Apartment for Sale in Palan Pur Patiya S...,Super Area,1150,Resale,1 out of 7,Resale,Unfurnished,1,Multistorey Apartment is located at the advant...,3304,3800000.0
70,2 BHK Apartment for Sale in Adajan Surat,Super Area,1200,Resale,3 out of 10,Resale,Furnished,2,2 BHK multistorey apartment is available for s...,3333,4000000.0
198,2 BHK Apartment for Sale in Adajan Surat,Super Area,960,Resale,3 out of 7,Resale,Semi-Furnished,NaN,NaN,4167,4000000.0
326,1 BHK Builder Floor for Sale in Saroli Surat,Carpet Area,12,Resale,3 out of 3,Resale,Unfurnished,2,NaN,33333,500000.0
360,"3 BHK Apartment for Sale in Mangalam Park, Pip...",Super Area,1420,Resale,1 out of 7,Resale,Semi-Furnished,Main Road,"3 BHK flat is available for sale. Facing Road,...",3169,4500000.0
368,Office Space for Sale in Lal Gate Surat,NaN,0,Resale,3 out of 5,Resale,NaN,NaN,NaN,NaN,3000000.0
417,2 BHK Apartment for Sale in Adajan Surat,Super Area,950,Resale,5 out of 5,Resale,Semi-Furnished,2,"east facing,semi furnished and located near lo...",3158,3000000.0
467,2 BHK Apartment for Sale in Nilkanth Park soci...,Super Area,1083,Resale,5 out of 5,Resale,Furnished,NaN,NaN,3509,3800000.0
471,1 BHK Apartment for Sale in Morabhagal Surat,NaN,0,Resale,13 out of 14,Resale,Unfurnished,NaN,NaN,NaN,1500000.0
537,"2 BHK Apartment for Sale in Aagam Navkar, Sach...",Super Area,570,Resale,4 out of 5,Resale,Unfurnished,2,NaN,3509,2000000.0


In [ ]:
pattern = r'^\d+\s*out\s*of\s*\d+$'

# Apply the condition to fill 'floor' with 'status' where 'status' matches the pattern and 'floor' doesn't match the pattern
df.loc[df['status'].str.match(pattern, na=False) & ~df['floor'].str.match(pattern, na=False), 'floor'] = df['status']

# Count how many rows are filled
filled_count = df['floor'].notna().sum()

# Display the filled rows and count
print(filled_count)


4480


#### if status is number out of number and floor is any other value append the number out of number  to floor column

In [ ]:
# Mask where status is 'number out of number'
mask = df['status'].astype(str).str.match(r'^\d+\s*out\s*of\s*\d+$')

# Mask where floor is not null
floor_not_null = df['floor'].notna()

# Combined mask: status is "number out of number" AND floor is not null
combined_mask = mask & floor_not_null

# Count how many
appended_count = combined_mask.sum()

print(f"Number of rows where 'status' was appended to 'floor': {appended_count}")


Number of rows where 'status' was appended to 'floor': 52


#### sound good as we will split them later

In [ ]:
status_counts = df['status'].value_counts()

print("\nCount of each unique value in ")
print(status_counts)


Count of each unique value in 
status
Ready to Move       3078
Poss. by Dec '24     155
Poss. by Dec '25     102
Resale                95
Poss. by Jul '24      78
                    ... 
240 X 180              1
Poss. by Mar '28       1
1200 X 3               1
144 X 105              1
Poss. by Nov '28       1
Name: count, Length: 138, dtype: int64


In [ ]:
import pandas as pd
import numpy as np

# Define the categorize_status function
def categorize_status(x):
    if pd.isna(x):
        return np.nan
    x = str(x)
    if x == "Ready to Move":
        return "Ready to Move"
    elif x.startswith("Poss. by"):
        return "Under Construction"
    elif x in ["New Property", "Resale", "Freehold", "Co-operative Society", "Power Of Attorney"]:
        return "Ready to Move"
    elif x == "Const. Age New Construction":
        return "Under Construction"
    else:
        return x  # Keep the original value if it doesn't match any condition

# Apply the function to the 'status' column
df['status'] = df['status'].apply(categorize_status)

# Count how many values are in 'Ready to Move' and 'Under Construction'
count_ready_to_move = df['status'].value_counts().get("Ready to Move", 0)
count_under_construction = df['status'].value_counts().get("Under Construction", 0)

print(f"Count of 'Ready to Move': {count_ready_to_move}")
print(f"Count of 'Under Construction': {count_under_construction}")


Count of 'Ready to Move': 3282
Count of 'Under Construction': 1118


In [ ]:
df.isna().sum()

,0
property_name,0
areaWithType,255
square_feet,0
transaction,27
status,1
floor,45
furnishing,340
facing,589
description,1371
price_per_sqft,368


In [ ]:
# Filter rows where 'status' is neither 'Under Construction' nor 'Ready to Move'
filtered_rows = df[~df['status'].isin(['Under Construction', 'Ready to Move'])]

# Display the filtered rows
filtered_rows


,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
22,2 BHK Apartment for Sale in Palan Pur Patiya S...,Super Area,1150,Resale,1 out of 7,1 out of 7,Unfurnished,1,Multistorey Apartment is located at the advant...,3304,3800000.0
40,Plot/Land for Sale in Narthan Surat,NaN,1055,Resale,21 X 50.2,Freehold,2,1,More than 7 international school running in th...,2275,2400000.0
70,2 BHK Apartment for Sale in Adajan Surat,Super Area,1200,Resale,3 out of 10,3 out of 10,Furnished,2,2 BHK multistorey apartment is available for s...,3333,4000000.0
179,Plot/Land for Sale in Olpad Sayan Road Surat,NaN,1413,Resale,120 X 105,Freehold,2,30 m,Residential plot is available for sale. It is ...,1699,2400000.0
198,2 BHK Apartment for Sale in Adajan Surat,Super Area,960,Resale,3 out of 7,3 out of 7,Semi-Furnished,NaN,NaN,4167,4000000.0
...,...,...,...,...,...,...,...,...,...,...,...
4058,Plot/Land for Sale in Piplod Surat,NaN,2025,Resale,144 X 126,Resale,No,NaN,NaN,12222,0.0
4082,"4 BHK Apartment for Sale in Unison Crest, Vesu...",Super Area,4604,New Property,4 out of 9,4 out of 9,Unfurnished,"Garden/Park, Pool, Main Road",ULTRA Luxurious 4 5 bhk flats in GD Goenka ca...,6851,0.0
4084,Plot/Land for Sale in Piplod Surat,NaN,4770,Resale,240 X 180,Freehold,3,2,plot. good. location. gated. Community. Mainta...,16667,0.0
4191,Plot/Land for Sale in Piplod Surat,NaN,3600,Resale,1200 X 3,Freehold,1,1,400 Vaar Open Plot On sale Piplod V I P Area O...,12778,0.0


In [ ]:
df.isna().sum()

,0
property_name,0
areaWithType,255
square_feet,0
transaction,27
status,1
floor,45
furnishing,340
facing,589
description,1371
price_per_sqft,368


In [ ]:
df[df['status'].isna()]


,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
950,House for Sale in Kamrej Surat,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2400000.0


will drop it

In [ ]:
before = df.shape[0]
df = df.dropna(subset=['status'])
after = df.shape[0]

print(f"Dropped {before - after} rows where status was NaN.")


Dropped 1 rows where status was NaN.


## floor

In [ ]:
floor_counts = df['floor'].value_counts()

print("\nCount of each unique value in ")
print(floor_counts)


Count of each unique value in 
floor
Resale          398
New Property    248
5 out of 13     212
7 out of 13     152
6 out of 13     129
               ... 
4 out of 20       1
2 out of 17       1
9 out of 17       1
14 out of 20      1
1 out of 18       1
Name: count, Length: 222, dtype: int64


In [ ]:
print(df['floor'].unique())

['5 out of 10' '6 out of 14' '5 out of 13' 'Unfurnished' 'New Property'
 '7 out of 10' '3 out of 14' '6 out of 13' '1 out of 1' '1 out of 13'
 '4 out of 7' '3 out of 3' '2 out of 6' '3 out of 5' '2 out of 5'
 '3 out of 12' '5 out of 5' '1 out of 7' 'Ground out of 1'
 'Lower Basement out of 7' '1 out of 5' '7 out of 14' '9 out of 14'
 '5 out of 12' '7 out of 13' 'Resale' '1 out of 2' '2 out of 10'
 'Freehold' '12 out of 14' '10 out of 14' '2 out of 12' '1 out of 4'
 '13 out of 14' '4 out of 14' '6 out of 10' '2 out of 4' '3 out of 4'
 '3 out of 10' '9 out of 13' 'Upper Basement out of 5' '1 out of 3'
 '4 out of 4' 'Ground out of 2' 'Lower Basement out of 2'
 'Ground out of 4' nan 'Ground out of 5' '4 out of 10' 'Ground out of 14'
 'Ground out of 6' '5 out of 6' '8 out of 9' '4 out of 5' '2 out of 7'
 '7 out of 7' 'Ground out of 3' '4 out of 8' '8 out of 14' '6 out of 7'
 '7 out of 11' 'Upper Basement out of 4' 'Furnished' 'Congo' '3'
 '2 out of 3' '3 out of 7' 'Ground' '1 out of 6' '9 o

In [ ]:
def modify_floor(floor):
    if isinstance(floor, str):
        # Check if the floor contains "Ground out of"
        if "Ground out of" in floor:
            return floor.replace("Ground out of", "0 out of")
        # Check if it's "Ground" alone
        elif "Ground" in floor:
            return "0 out of 0"
        # Check if the floor contains "Upper Basement out of"
        elif "Upper Basement out of" in floor:
            return floor.replace("Upper Basement out of", "-1 out of")
        # Check if the floor contains "Lower Basement out of"
        elif "Lower Basement out of" in floor:
            return floor.replace("Lower Basement out of", "-2 out of")
        else:
            return floor
    return floor

df['floor'] = df['floor'].apply(modify_floor)

modified_floor = df[df['floor'].str.contains('out of', na=False)]


In [ ]:
# Filter rows where 'floor' does not match the pattern "number out of number" allowing negative numbers
non_matching_floor = df[~df['floor'].astype(str).str.match(r'^-?\d+\s*out\s*of\s*-?\d+$', na=False)]

print(f"Number of rows where floor is not 'number out of number': {non_matching_floor.shape[0]}")


Number of rows where floor is not 'number out of number': 826


In [ ]:
non_matching_floor


,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
4,"2 BHK Apartment for Sale in Orchid Fantasia, P...",Super Area,1250,New Property,Ready to Move,Unfurnished,2,2,"Multistorey Apartment for Sale in Palanpur, Su...",3600,4500000.0
5,"2 BHK Apartment for Sale in Anand Aspire, Jaha...",Super Area,1265,New Property,Under Construction,New Property,Unfurnished,2,Multistorey Apartment for Sale in Jahangirabad...,3411,4320000.0
6,3 BHK Villa for Sale in Dindoli Surat,Carpet Area,1404,New Property,Under Construction,New Property,Unfurnished,Main Road,contraction,NaN,4210000.0
8,"2 BHK Apartment for Sale in Orchid Gardenia, P...",Super Area,1180,New Property,Ready to Move,Unfurnished,2,2,"Multistorey Apartment for Sale in Palanpur, Su...",3751,4430000.0
26,2 BHK Apartment for Sale in Palanpur Surat,Carpet Area,650,New Property,Ready to Move,New Property,Unfurnished,Main Road,LUXURY NOW AFFORDABLE 2 BHK FLAT.Two Separate ...,4150,4650000.0
...,...,...,...,...,...,...,...,...,...,...,...
4377,4 BHK Apartment for Sale in Bblossom The Presi...,Super Area,3500,New Property,Under Construction,New Property,Unfurnished,4,"Multistorey Apartment for Sale in VIP Road, Su...",5450,0.0
4384,4 BHK Apartment for Sale in Bblossom The Presi...,Super Area,3676,New Property,Under Construction,New Property,Unfurnished,4,"Multistorey Apartment for Sale in VIP Road, Su...",5450,0.0
4390,4 BHK Apartment for Sale in Bblossom The Presi...,Super Area,3700,New Property,Under Construction,New Property,Unfurnished,4,"Multistorey Apartment for Sale in VIP Road, Su...",5450,0.0
4396,4 BHK Apartment for Sale in Bblossom The Presi...,Super Area,4599,New Property,Under Construction,New Property,Unfurnished,4,"Multistorey Apartment for Sale in VIP Road, Su...",5450,0.0


In [ ]:
# If 'description' contains "Villa" (case insensitive) and 'floor' is missing or anything, set 'floor' to "1 out of 1"
df.loc[df['description'].str.contains('villa', case=False, na=False), 'floor'] = '1 out of 1'


In [ ]:
# Filter rows where 'description' contains the word 'floors' and 'floor' is not in the 'number out of number' format
filtered_rows = df[
    df['description'].str.contains('floors', case=False, na=False) &
    ~df['floor'].astype(str).str.match(r'^\d+\s*out\s*of\s*\d+$', na=False)
]

filtered_rows


,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
132,2 BHK House for Sale in Bardoli Surat,Super Area,1000,Resale,Ready to Move,NaN,NaN,NaN,This Residential House of total floors is me...,4500,4500000.0
154,2 BHK House for Sale in Kamrej Surat,Super Area,90,Resale,Ready to Move,Resale,Semi-Furnished,NaN,Residential House is made upto 1 floors. The...,5556,4500000.0
285,2 BHK House for Sale in Velanja Surat,Super Area,140,Resale,Ready to Move,Unfurnished,1,NaN,This Residential House of total floors is me...,2778,3500000.0
303,1 BHK House for Sale in Kosamba Surat,Super Area,480,Resale,Ready to Move,Resale,Unfurnished,NaN,This spacious Residential House has total 1 f...,1875,900000.0
361,Plot/Land for Sale in Pandesara Surat,NaN,420,Resale,Ready to Move,1,NaN,NaN,Embark on a journey of endless possibilities w...,3333,1400000.0
731,1 BHK House for Sale in Kadodara Surat,Super Area,432,Resale,Ready to Move,Resale,Unfurnished,NaN,This Residential House is built over 1 floors...,3266,1410000.0
823,1 BHK House for Sale in Palanpur Gam Surat,Super Area,630,Resale,Ready to Move,Resale,Semi-Furnished,1,This Residential House is built over 1 floors...,7460,4700000.0
975,2 BHK House for Sale in Amroli Surat,Super Area,576,Resale,Ready to Move,Resale,Semi-Furnished,1,This Residential House is built over 1 floors...,6076,3500000.0
1727,7 BHK House for Sale in Palanpur Jakatnaka Surat,Super Area,2100,Resale,Ready to Move,Resale,Semi-Furnished,6,Residential House is made upto 2 floors. The...,4286,9000000.0
1802,2 BHK House for Sale in Nanavat Surat,Super Area,864,Resale,Ready to Move,-2 out of 5,Semi-Furnished,North,This Residential House is built over 2 floors...,9259,8000000.0


In [ ]:
import re
import numpy as np
import pandas as pd

# Function to extract floor from description based on "is made upto number floors"
def extract_floor_from_description(description):
    # Search for the pattern "is made upto number floors"
    match = re.search(r'is made upto (\d+)\s*floors?', str(description), re.IGNORECASE)
    if match:
        number_of_floors = match.group(1)  # Extract the number of floors
        return f"{number_of_floors} out of {number_of_floors}"  # Return as "number out of number"
    return np.nan  # Return NaN if no match is found

# Function to extract top floor number from description
def extract_top_floor_from_description(description):
    # Search for the pattern "on the top floor of a number-floor"
    match = re.search(r'on the top floor of a (\d+)-floor', str(description), re.IGNORECASE)
    if match:
        top_floor = match.group(1)  # Extract the number of the top floor
        return f"{top_floor} out of {top_floor}"  # Return as "number out of number"
    return np.nan  # Return NaN if no match is found

# Function to combine both extraction logic
def extract_floor(description):
    # Try to extract from the first pattern ("is made upto number floors")
    floor = extract_floor_from_description(description)
    if not pd.isna(floor):  # If a match is found for the first pattern
        return floor

    # If no match for the first pattern, try extracting from the second pattern ("on the top floor of a number-floor")
    return extract_top_floor_from_description(description)

# Apply the combined function to the 'description' column to fill the 'floor' column only where needed
df['floor'] = df.apply(lambda row: extract_floor(row['description']) if pd.isna(row['floor']) else row['floor'], axis=1)

# Display the updated rows where the floor was updated
updated_rows = df[df['floor'].notna()]
print(updated_rows)


                                          property_name areaWithType  \
0             2 BHK Apartment for Sale in Dindoli Surat  Carpet Area   
1              2 BHK Apartment for Sale in Althan Surat   Super Area   
2             2 BHK Apartment for Sale in Pal Gam Surat   Super Area   
3        2 BHK Apartment for Sale in Jahangirabad Surat  Carpet Area   
4     2 BHK Apartment for Sale in Orchid Fantasia, P...   Super Area   
...                                                 ...          ...   
4520  6 BHK Apartment for Sale in Millionaires Lifes...  Carpet Area   
4521  4 BHK Apartment for Sale in Savan Superia, Alt...   Super Area   
4522  5 BHK Apartment for Sale in Roongta Green Vall...  Carpet Area   
4523  6 BHK Apartment for Sale in Cellestial Dreams,...  Carpet Area   
4524  4 BHK Apartment for Sale in Roongta Green Vall...   Super Area   

      square_feet   transaction              status        floor  \
0             644  New Property  Under Construction  5 out of 10   

In [ ]:
# Count the rows where the 'floor' column does not match the pattern "number out of number"
non_matching_floor_count = df[~df['floor'].astype(str).str.match(r'^-?\d+\s*out\s*of\s*-?\d+$', na=False)].shape[0]

# Display the count
print(f"Count of rows where 'floor' is not in the 'number out of number' format: {non_matching_floor_count}")


Count of rows where 'floor' is not in the 'number out of number' format: 783


In [ ]:
# Get the unique values in the 'floor' column that do not match the pattern "number out of number"
non_matching_floor_categories = df[~df['floor'].astype(str).str.match(r'^-?\d+\s*out\s*of\s*-?\d+$', na=False)]['floor'].unique()

# Display the categories
print("Categories in 'floor' column that do not match the 'number out of number' format:")
print(non_matching_floor_categories)


Categories in 'floor' column that do not match the 'number out of number' format:
['Unfurnished' 'New Property' 'Resale' 'Freehold' nan 'Furnished' 'Congo'
 '3' '1' '2' 'No' 'Yes' 'Semi-Furnished' 'Other' 'The Polaris Avenue'
 'Co-operative Society' 'Om Residency' 'Milestone Regalia' '5']


In [ ]:
filtered_rows = df[df['floor'].isin(['3', '1', '2'])]

filtered_rows


,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
189,Plot/Land for Sale in Silver park plot no 1 ...,NaN,2493,Resale,Ready to Move,3,NaN,NaN,NaN,309,770000.0
193,Plot/Land for Sale in Palsana Surat,NaN,648,Resale,Ready to Move,3,NaN,NaN,NaN,3086,2000000.0
204,Plot/Land for Sale in Kim Surat,NaN,650,Resale,Ready to Move,3,NaN,NaN,NaN,1846,1200000.0
238,Plot/Land for Sale in Sayan Surat,NaN,860,Resale,Ready to Move,3,NaN,NaN,NaN,1744,1500000.0
254,Plot/Land for Sale in Masma Village Surat,NaN,927,Resale,Ready to Move,3,NaN,NaN,NaN,1942,1800000.0
324,1 BHK House for Sale in Jolva Surat,NaN,0,NaN,2,1,NaN,NaN,NaN,NaN,1200000.0
346,Plot/Land for Sale in Maroli Surat,NaN,963,Resale,Ready to Move,2,NaN,NaN,NaN,987,950000.0
361,Plot/Land for Sale in Pandesara Surat,NaN,420,Resale,Ready to Move,1,NaN,NaN,Embark on a journey of endless possibilities w...,3333,1400000.0
422,Plot/Land for Sale in Masma Village Surat,NaN,1251,Resale,Ready to Move,3,NaN,NaN,NaN,2798,3500000.0
425,Plot/Land for Sale in Hajira Surat,NaN,1287,Resale,Ready to Move,1,NaN,NaN,NaN,3885,5000000.0


if ['3', '1', '2'] is plot or land will create feature engineering

In [ ]:
df.isna().sum()

,0
property_name,0
areaWithType,254
square_feet,0
transaction,26
status,0
floor,43
furnishing,339
facing,588
description,1370
price_per_sqft,367


In [ ]:
# Count the rows where 'floor' is 'New Property' or 'Resale'
count_new_property_resale = df[df['floor'].isin(['New Property', 'Resale'])].shape[0]

# Display the count
print(f"Count of rows where 'floor' is 'New Property' or 'Resale': {count_new_property_resale}")


Count of rows where 'floor' is 'New Property' or 'Resale': 607


In [ ]:
# Fill 'transaction' with the value from 'floor' where 'floor' is 'New Property' or 'Resale'
df.loc[df['floor'].isin(['New Property', 'Resale']), 'transaction'] = df['floor']


In [ ]:

def standardize_floor(value):
    value = str(value).strip()
    if re.match(r'^Ground\s+out\s+of\s+\d+$', value, re.IGNORECASE):
        number = re.search(r'\d+', value).group()
        return f'0 out of {number}'
    elif re.match(r'^Lower Basement\s+out\s+of\s+\d+$', value, re.IGNORECASE):
        number = re.search(r'\d+', value).group()
        return f'-1 out of {number}'
    elif re.match(r'^Upper Basement\s+out\s+of\s+\d+$', value, re.IGNORECASE):
        number = re.search(r'\d+', value).group()
        return f'-2 out of {number}'
    elif value.lower() == 'ground':
        return '0'
    else:
        return value

# Apply the function
df['floor'] = df['floor'].apply(standardize_floor)

# See some examples
print(df['floor'].sample(10))


1840     1 out of 10
518      -2 out of 4
3472     6 out of 13
2384     5 out of 13
2947     8 out of 14
396       0 out of 2
693       4 out of 4
3058    New Property
4065     5 out of 13
3065      1 out of 1
Name: floor, dtype: object


## furnishing

In [ ]:
print(df['furnishing'].unique())

['Unfurnished' 'Semi-Furnished' '2' '1' 'Furnished' nan '3' '1 Covered,'
 '2 Covered' '1 Covered' '26 Covered' 'No' '6' '> 10' '4' '10 Open'
 '5 Covered,' '7 m' '30 m' '50 Covered,' 'Main Road' '1 Open' '12 m' 'Yes'
 '2 Covered,' 'Congo' 'Vaishnodevi Ideal Homes' '100 Covered' 'West'
 '5 Covered' '5' '50 Open' '20 m' '20 Open' '10 m' '3 Covered' '2 Open'
 '6 m' '15 m' '10 Covered,']


In [ ]:
def fill_facing(row):
    if pd.isna(row['facing']):
        if row['furnishing'] in ['East','West','North', 'North - East', 'South -West', 'South - East', 'North - West']:
            return row['furnishing']
    return row['facing']

df['facing'] = df.apply(fill_facing, axis=1)


In [ ]:
furnishing_counts = df['furnishing'].value_counts()

print("\nCount of each unique value in ")
print(furnishing_counts)


Count of each unique value in 
furnishing
Unfurnished                2726
Semi-Furnished              582
Furnished                   526
1                           109
2                            80
3                            37
1 Covered                    25
1 Open                       15
No                           14
4                             9
2 Covered                     8
1 Covered,                    6
2 Open                        5
5                             5
2 Covered,                    5
Congo                         3
100 Covered                   2
6 m                           2
3 Covered                     2
Yes                           2
> 10                          2
30 m                          2
6                             2
12 m                          1
Main Road                     1
7 m                           1
50 Covered,                   1
5 Covered,                    1
10 Open                       1
26 Covered                   

In [ ]:
df['furnishing'] = df['furnishing'].astype(str)

df['furnishing'] = df['furnishing'].replace('1', 'Furnished')


In [ ]:
df['furnishing'] = df['furnishing'].replace('Yes', 'Furnished')

In [ ]:
df['furnishing'] = df['furnishing'].astype(str)

df['furnishing'] = df['furnishing'].replace('No', 'Unfurnished')

In [ ]:
df['furnishing'].value_counts()

,count
furnishing,
Unfurnished,2740
Furnished,637
Semi-Furnished,582
nan,339
2,80
3,37
1 Covered,25
1 Open,15
4,9


In [ ]:
# Define the valid furnishing types
valid_furnishings = ['Unfurnished', 'Furnished', 'Semi-Furnished']

# Filter rows where furnishing is NOT in the valid list
invalid_furnishings = df[~df['furnishing'].isin(valid_furnishings)]

# Display
invalid_furnishings


,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price
4,"2 BHK Apartment for Sale in Orchid Fantasia, P...",Super Area,1250,New Property,Ready to Move,Unfurnished,2,2,"Multistorey Apartment for Sale in Palanpur, Su...",3600,4500000.0
8,"2 BHK Apartment for Sale in Orchid Gardenia, P...",Super Area,1180,New Property,Ready to Move,Unfurnished,2,2,"Multistorey Apartment for Sale in Palanpur, Su...",3751,4430000.0
16,Shop for Sale in Moti Begumwadi Surat,Carpet Area,180,Resale,Ready to Move,3 out of 3,nan,NaN,"4056,Hari om Textile Market, Ring Road, Near R...",14728,2650000.0
24,Shop for Sale in Ghod Dod Road Surat,Carpet Area,350,Resale,Ready to Move,-2 out of 7,nan,NaN,Commercial Shop is available for Sale in Gho...,12857,4500000.0
37,Plot/Land for Sale in Bhatpore Surat,NaN,120,Resale,Ready to Move,Resale,2,Garden/Park,120 Sq-ft Residential Plot is available for Sa...,35000,4200000.0
...,...,...,...,...,...,...,...,...,...,...,...
4282,Showroom for Sale in Vesu Canal Road Surat,Carpet Area,800,New Property,Under Construction,0 out of 6,2 Covered,NaN,NaN,25000,0.0
4300,Plot/Land for Sale in Piplod Surat,NaN,1683,Resale,144 X 105,Co-operative Society,2,8 m,"near temple, school, multiplex,",12181,0.0
4306,Plot/Land for Sale in Vesu Surat,NaN,1260,Resale,Ready to Move,Resale,2,15 m,NaN,15079,0.0
4317,Plot/Land for Sale in Vesu Surat,NaN,2700,Resale,Ready to Move,Resale,2,15 m,NaN,14444,0.0


In [ ]:
df.isna().sum()

,0
property_name,0
areaWithType,254
square_feet,0
transaction,26
status,0
floor,0
furnishing,0
facing,588
description,1370
price_per_sqft,367


filling missing price_per_sqft

In [ ]:
# Fill missing price_per_sqft with price / square_feet
mask = df['price_per_sqft'].isna() & df['price'].notna() & df['square_feet'].notna()

df.loc[mask, 'price_per_sqft'] = df.loc[mask, 'price'] / df.loc[mask, 'square_feet']


In [ ]:
filled_count = mask.sum()

print(f"Number of rows where 'price_per_sqft' was filled: {filled_count}")


Number of rows where 'price_per_sqft' was filled: 367


In [ ]:
furnishing_types = ['Unfurnished', 'Furnished', 'Semi-Furnished']

# Function to detect and correct furnishing
def correct_furnishing_from_description(row):
    if row['furnishing'] not in furnishing_types:
        description = str(row['description']).lower()
        for f_type in furnishing_types:
            if re.search(f_type.lower().replace('-', '[- ]?'), description):  # Handle both hyphen and space variations
                row['furnishing'] = f_type
                break
    return row

df = df.apply(correct_furnishing_from_description, axis=1)

In [ ]:
valid_furnishings = ['Unfurnished', 'Furnished', 'Semi-Furnished']

def fix_furnishing(row):
    if row['floor'] in valid_furnishings and row['furnishing'] not in valid_furnishings:
        row['furnishing'] = row['floor']  # Move the value from floor to furnishing
    return row

df = df.apply(fix_furnishing, axis=1)


In [ ]:
valid_values = ['Furnished', 'Unfurnished', 'Semi-Furnished']
df['furnishing'] = df['furnishing'].apply(lambda x: x if x in valid_values else np.nan)

print("\nUnique values after replacement:")
print(df['furnishing'].unique())


Unique values after replacement:
['Unfurnished' 'Semi-Furnished' 'Furnished' nan]


## facing


In [ ]:
print(df['facing'].unique())


['West' 'South -West' 'East' '2' 'Main Road' nan 'North - East'
 'South - East' '1' 'Freehold' 'Garden/Park' '3' 'North - West' 'South'
 '6' 'Co-operative Society' 'Garden/Park, Main Road' '1 Covered' 'No'
 '30 m' 'North' 'Varni Siddheshwar Heights' '5' 'Salasar Icon'
 'Green Valley' 'Shyam enclave' 'Rajhans Platinum' 'Vitthal Bunglows'
 '1 Open' 'Royal Nest' 'Rajhans Wings' '2 Open'
 'Gordhan Green Valley Mangalam Park Bldg S' 'Apt Swaminagar Society'
 'Swagat Clifton' 'Akash' 'Green Arcade Phase 1 And 2' 'Satyam Tower' '4'
 'Suryanjali Residency' 'GHB Mukhya Mantri Gruh Yojana' 'Vibrant Eco Park'
 'Madhav Residency' 'Shiv Drashti Residency' 'Gruham Luxuria'
 'SHANKHESHWER COMPLAX' 'JT Stuti Highland' 'Times Galaxy' '100 m'
 'Savani Prayosha Pride' 'Shayona Janki Residency' 'Sundaram Residency'
 '6 m' 'Silicon Palm' 'Raj Abhishek City Homes' 'Dhvani Royal Residency'
 'Residency' 'Pan Sharanam' 'Gruham Royal Park' 'Rajhans Swapna'
 'Panchtatva Residency' 'Manvay Residency' 'Prayosha Sh

In [ ]:
df['facing'] = df['facing'].replace('South -West', 'South - West')


In [ ]:
valid_values = [
    'North', 'North - East', 'East', 'South - East',
    'South', 'South - West', 'West', 'North - West'
]

df['facing'] = df['facing'].apply(lambda x: x if x in valid_values else np.nan)

print("\nUnique values after replacement:")
print(df['facing'].unique())


Unique values after replacement:
['West' 'South - West' 'East' nan 'North - East' 'South - East'
 'North - West' 'South' 'North']


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4524 entries, 0 to 4524
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   property_name   4524 non-null   object 
 1   areaWithType    4270 non-null   object 
 2   square_feet     4524 non-null   int64  
 3   transaction     4498 non-null   object 
 4   status          4524 non-null   object 
 5   floor           4524 non-null   object 
 6   furnishing      3993 non-null   object 
 7   facing          2382 non-null   object 
 8   description     3154 non-null   object 
 9   price_per_sqft  4524 non-null   object 
 10  price           4524 non-null   float64
dtypes: float64(1), int64(1), object(9)
memory usage: 553.2+ KB


# Golden Layer

In [ ]:
gdf = df.copy()

## Property Name

In [ ]:
gdf['property_name'].unique()


array(['2 BHK Apartment for Sale in Dindoli Surat',
       '2 BHK Apartment for Sale in Althan Surat',
       '2 BHK Apartment for Sale in Pal Gam Surat', ...,
       '5 BHK Apartment for Sale in Sangini Arise, Bharthana Surat',
       '5 BHK Apartment for Sale in D and M Elysium Avenue, Piplod Surat',
       '6 BHK Apartment for Sale in Millionaires Lifestyle, Vesu Surat'],
      dtype=object)

In [ ]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4524 entries, 0 to 4524
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   property_name   4524 non-null   object 
 1   areaWithType    4270 non-null   object 
 2   square_feet     4524 non-null   int64  
 3   transaction     4498 non-null   object 
 4   status          4524 non-null   object 
 5   floor           4524 non-null   object 
 6   furnishing      3993 non-null   object 
 7   facing          2382 non-null   object 
 8   description     3154 non-null   object 
 9   price_per_sqft  4524 non-null   object 
 10  price           4524 non-null   float64
dtypes: float64(1), int64(1), object(9)
memory usage: 553.2+ KB


In [ ]:
# Function to extract only bhk, property_type, locality, and city
def extract_main_features(name):
    name = str(name)

    # Extract BHK
    bhk_match = re.search(r'(\d+)\s*BHK', name, re.IGNORECASE)
    bhk = int(bhk_match.group(1)) if bhk_match else None

    # Extract property type (Apartment, Villa, Plot, etc.)
    type_match = re.search(r'BHK\s*(.*?)\s*for sale', name, re.IGNORECASE)
    property_type = type_match.group(1).strip() if type_match else None

    # Extract locality (after "for Sale in")
    locality = None
    city = None

    area_match = re.search(r'for sale in (.+)', name, re.IGNORECASE)
    if area_match:
        area_info = area_match.group(1).strip()
        parts = area_info.split()
        if len(parts) > 1:
            locality = " ".join(parts[:-1])
            city = parts[-1]
        else:
            locality = area_info
            city = "Surat"  # default

    return pd.Series({
        'bhk': bhk,
        'property_type': property_type,
        'locality': locality,
        'city': city if city else "Surat"
    })

gdf_extracted = gdf['property_name'].apply(extract_main_features)

gdf = pd.concat([gdf, gdf_extracted], axis=1)



In [ ]:
gdf.head()

,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price,bhk,property_type,locality,city
0,2 BHK Apartment for Sale in Dindoli Surat,Carpet Area,644,New Property,Under Construction,5 out of 10,Unfurnished,West,"Luxury project with basement parking, Solar ro...",2891,3380000.0,2.0,Apartment,Dindoli,Surat
1,2 BHK Apartment for Sale in Althan Surat,Super Area,1278,New Property,Under Construction,6 out of 14,Unfurnished,South - West,2 And 3 BHK Luxurious Flat for Sell In New Alt...,3551,4540000.0,2.0,Apartment,Althan,Surat
2,2 BHK Apartment for Sale in Pal Gam Surat,Super Area,1173,Resale,Ready to Move,5 out of 13,Semi-Furnished,East,This affordable 2 BHK flat is situated along a...,3800,4460000.0,2.0,Apartment,Pal Gam,Surat
3,2 BHK Apartment for Sale in Jahangirabad Surat,Carpet Area,700,New Property,Ready to Move,6 out of 14,Unfurnished,East,2 BHK Flat For sell IN Jahangirabad Prime Loca...,3966,4700000.0,2.0,Apartment,Jahangirabad,Surat
4,"2 BHK Apartment for Sale in Orchid Fantasia, P...",Super Area,1250,New Property,Ready to Move,Unfurnished,Unfurnished,NaN,"Multistorey Apartment for Sale in Palanpur, Su...",3600,4500000.0,2.0,Apartment,"Orchid Fantasia, Palanpur",Surat


## Transacion Type Standardizaion

In [ ]:
print(gdf['transaction'].unique())


['New Property' 'Resale' nan]


In [ ]:
# Filter rows where 'transaction' is NaN
nan_transaction_rows = gdf[gdf['transaction'].isna()]

# Display the rows
nan_transaction_rows

,property_name,areaWithType,square_feet,transaction,status,floor,furnishing,facing,description,price_per_sqft,price,bhk,property_type,locality,city
136,1 BHK Apartment for Sale in Vesu Surat,Carpet Area,500,NaN,Ready to Move,4 out of 8,Unfurnished,NaN,"1 BHK, Multistorey Apartment is available for ...",3200.0,1600000.0,1.0,Apartment,Vesu,Surat
171,1 BHK Apartment for Sale in Thane Dombivali Surat,Carpet Area,405,NaN,Ready to Move,Furnished,Furnished,NaN,NaN,8148.148148,3300000.0,1.0,Apartment,Thane Dombivali,Surat
324,1 BHK House for Sale in Jolva Surat,NaN,0,NaN,2,1,NaN,NaN,NaN,inf,1200000.0,1.0,House,Jolva,Surat
440,2 BHK Villa for Sale in Vkhome near madav resi...,Carpet Area,648,NaN,Ready to Move,Unfurnished,Unfurnished,NaN,NaN,3703.703704,2400000.0,2.0,Villa,Vkhome near madav residency tatithiya gam kado...,Surat
512,3 BHK Villa for Sale in Rushi nagar society Go...,Carpet Area,1100,NaN,Ready to Move,Semi-Furnished,Semi-Furnished,NaN,NaN,2272.727273,2500000.0,3.0,Villa,Rushi nagar society Godadar,Surat
570,"2 BHK Apartment for Sale in Vinayak Enclave, D...",Carpet Area,897,NaN,Ready to Move,2 out of 5,Furnished,NaN,NaN,2564.102564,2300000.0,2.0,Apartment,"Vinayak Enclave, Dindoli",Surat
681,"2 BHK Apartment for Sale in Palanpur ,Canal ro...",Carpet Area,1246,NaN,Ready to Move,4 out of 12,Semi-Furnished,NaN,NaN,4012.841091,5000000.0,2.0,Apartment,"Palanpur ,Canal road",Surat
734,2 BHK Apartment for Sale in Gardenview complex...,Carpet Area,1190,NaN,Ready to Move,7 out of 7,Semi-Furnished,NaN,NaN,3361.344538,4000000.0,2.0,Apartment,Gardenview complex honeypark Road adajan,Surat
772,2 BHK Apartment for Sale in Thane Surat,Carpet Area,780,NaN,Ready to Move,Other,Semi-Furnished,NaN,Multistorey apartment is available for sale. I...,4487.179487,3500000.0,2.0,Apartment,Thane,Surat
895,"2 BHK Apartment for Sale in JAHANGIRABAD, UGAT...",Carpet Area,512,NaN,Ready to Move,2 out of 13,Semi-Furnished,NaN,NaN,3027.34375,1550000.0,2.0,Apartment,"JAHANGIRABAD, UGAT BHESAN ROAD",Surat


In [ ]:
def standardize_transaction_type(x):
    if isinstance(x, str):
        x = x.strip().title()
        if 'New' in x:
            return 'New Property'
        elif 'Resale' in x:
            return 'Resale'
    return None

gdf['transaction_standardized'] = gdf['transaction'].apply(standardize_transaction_type)


In [ ]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4524 entries, 0 to 4524
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   property_name             4524 non-null   object 
 1   areaWithType              4270 non-null   object 
 2   square_feet               4524 non-null   int64  
 3   transaction               4498 non-null   object 
 4   status                    4524 non-null   object 
 5   floor                     4524 non-null   object 
 6   furnishing                3993 non-null   object 
 7   facing                    2382 non-null   object 
 8   description               3154 non-null   object 
 9   price_per_sqft            4524 non-null   object 
 10  price                     4524 non-null   float64
 11  bhk                       3889 non-null   float64
 12  property_type             3889 non-null   object 
 13  locality                  4524 non-null   object 
 14  city         

In [ ]:
gdf.drop(columns=['transaction'], inplace=True)

In [ ]:
gdf.head()

,property_name,areaWithType,square_feet,status,floor,furnishing,facing,description,price_per_sqft,price,bhk,property_type,locality,city,transaction_standardized
0,2 BHK Apartment for Sale in Dindoli Surat,Carpet Area,644,Under Construction,5 out of 10,Unfurnished,West,"Luxury project with basement parking, Solar ro...",2891,3380000.0,2.0,Apartment,Dindoli,Surat,New Property
1,2 BHK Apartment for Sale in Althan Surat,Super Area,1278,Under Construction,6 out of 14,Unfurnished,South - West,2 And 3 BHK Luxurious Flat for Sell In New Alt...,3551,4540000.0,2.0,Apartment,Althan,Surat,New Property
2,2 BHK Apartment for Sale in Pal Gam Surat,Super Area,1173,Ready to Move,5 out of 13,Semi-Furnished,East,This affordable 2 BHK flat is situated along a...,3800,4460000.0,2.0,Apartment,Pal Gam,Surat,Resale
3,2 BHK Apartment for Sale in Jahangirabad Surat,Carpet Area,700,Ready to Move,6 out of 14,Unfurnished,East,2 BHK Flat For sell IN Jahangirabad Prime Loca...,3966,4700000.0,2.0,Apartment,Jahangirabad,Surat,New Property
4,"2 BHK Apartment for Sale in Orchid Fantasia, P...",Super Area,1250,Ready to Move,Unfurnished,Unfurnished,NaN,"Multistorey Apartment for Sale in Palanpur, Su...",3600,4500000.0,2.0,Apartment,"Orchid Fantasia, Palanpur",Surat,New Property


In [ ]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4524 entries, 0 to 4524
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   property_name             4524 non-null   object 
 1   areaWithType              4270 non-null   object 
 2   square_feet               4524 non-null   int64  
 3   status                    4524 non-null   object 
 4   floor                     4524 non-null   object 
 5   furnishing                3993 non-null   object 
 6   facing                    2382 non-null   object 
 7   description               3154 non-null   object 
 8   price_per_sqft            4524 non-null   object 
 9   price                     4524 non-null   float64
 10  bhk                       3889 non-null   float64
 11  property_type             3889 non-null   object 
 12  locality                  4524 non-null   object 
 13  city                      4524 non-null   object 
 14  transaction_s

In [ ]:
print(gdf['price_per_sqft'].unique())


['2891' '3551' '3800' ... '8512' '8200' '5756']


In [ ]:
gdf['price_per_sqft'] = pd.to_numeric(gdf['price_per_sqft'], errors='coerce')


## Standardized Total Price

In [ ]:
print(gdf['price'].unique())


[ 3380000.  4540000.  4460000.  4700000.  4500000.  4320000.  4210000.
  4410000.  4430000.  4000000.  4400000.  1250000.  5000000.  3580000.
  2650000.  4200000.  4300000.  2000000.  3700000.  2900000.  3800000.
  1300000.  4650000.  4160000.  3350000.  4250000.  4800000.  3200000.
  3500000.  4620000.  3850000.  2300000.  2400000.  2800000.  3570000.
  3650000.  3600000.  4360000.  2500000.  4100000.  2850000.   900000.
  3000000.  3950000.  2150000.  1050000.  1800000.  1000000.  2350000.
  1630000.  1350000.  4900000.  2100000.  3900000.  1600000.  2700000.
  1500000.  3990000.   750000.  1280000.   950000.  3150000.  4600000.
   600000.  3400000.  2200000.  1900000.   850000.  1100000.  1650000.
  1700000.  1400000.  3300000.  2050000.  2170000.  2750000.  1200000.
  2600000.   770000.  4480000.  3490000.  1950000.  1850000.  4850000.
  4990000.  2670000.  1820000.  2450000.  4610000.  1420000.  4750000.
   930000.   500000.   890000.  1450000.  3760000.  2550000.  2950000.
   150

Already did in silver layer

In [ ]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4524 entries, 0 to 4524
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   property_name             4524 non-null   object 
 1   areaWithType              4270 non-null   object 
 2   square_feet               4524 non-null   int64  
 3   status                    4524 non-null   object 
 4   floor                     4524 non-null   object 
 5   furnishing                3993 non-null   object 
 6   facing                    2382 non-null   object 
 7   description               3154 non-null   object 
 8   price_per_sqft            4524 non-null   float64
 9   price                     4524 non-null   float64
 10  bhk                       3889 non-null   float64
 11  property_type             3889 non-null   object 
 12  locality                  4524 non-null   object 
 13  city                      4524 non-null   object 
 14  transaction_s

In [ ]:
df['square_feet'].unique()

array([ 644, 1278, 1173, ..., 2545, 5175, 2620])

## Floor Number & Total Floors

In [ ]:
gdf.head()

,property_name,areaWithType,square_feet,status,floor,furnishing,facing,description,price_per_sqft,price,bhk,property_type,locality,city,transaction_standardized
0,2 BHK Apartment for Sale in Dindoli Surat,Carpet Area,644,Under Construction,5 out of 10,Unfurnished,West,"Luxury project with basement parking, Solar ro...",2891.0,3380000.0,2.0,Apartment,Dindoli,Surat,New Property
1,2 BHK Apartment for Sale in Althan Surat,Super Area,1278,Under Construction,6 out of 14,Unfurnished,South - West,2 And 3 BHK Luxurious Flat for Sell In New Alt...,3551.0,4540000.0,2.0,Apartment,Althan,Surat,New Property
2,2 BHK Apartment for Sale in Pal Gam Surat,Super Area,1173,Ready to Move,5 out of 13,Semi-Furnished,East,This affordable 2 BHK flat is situated along a...,3800.0,4460000.0,2.0,Apartment,Pal Gam,Surat,Resale
3,2 BHK Apartment for Sale in Jahangirabad Surat,Carpet Area,700,Ready to Move,6 out of 14,Unfurnished,East,2 BHK Flat For sell IN Jahangirabad Prime Loca...,3966.0,4700000.0,2.0,Apartment,Jahangirabad,Surat,New Property
4,"2 BHK Apartment for Sale in Orchid Fantasia, P...",Super Area,1250,Ready to Move,Unfurnished,Unfurnished,NaN,"Multistorey Apartment for Sale in Palanpur, Su...",3600.0,4500000.0,2.0,Apartment,"Orchid Fantasia, Palanpur",Surat,New Property


In [ ]:
import pandas as pd
import re

# Function to extract floor information
def extract_floor_info(floor_str):
    if isinstance(floor_str, str):
        match = re.match(r'(\d+)\s+out of\s+(\d+)', floor_str)  # Match the "X out of Y" pattern
        if match:
            floor_number = int(match.group(1))
            total_floors = int(match.group(2))
            return pd.Series([floor_number, total_floors])

        # If there is only one number (no "out of")
        match_single = re.match(r'(\d+)', floor_str)
        if match_single:
            floor_number = int(match_single.group(1))
            return pd.Series([floor_number, floor_number])  # Both floor_number and total_floors are the same

    return pd.Series([None, None])  # Return None if no match

# Assuming 'gdf' is your DataFrame and 'floor' is the column containing the floor information
gdf[['floor_number', 'total_floors']] = gdf['floor'].apply(extract_floor_info)

# Display the updated DataFrame with the new columns
print(gdf[['floor', 'floor_number', 'total_floors']])


            floor  floor_number  total_floors
0     5 out of 10           5.0          10.0
1     6 out of 14           6.0          14.0
2     5 out of 13           5.0          13.0
3     6 out of 14           6.0          14.0
4     Unfurnished           NaN           NaN
...           ...           ...           ...
4520  5 out of 12           5.0          12.0
4521  5 out of 16           5.0          16.0
4522  7 out of 13           7.0          13.0
4523  7 out of 18           7.0          18.0
4524  3 out of 12           3.0          12.0

[4524 rows x 3 columns]


In [ ]:
gdf.head()

,property_name,areaWithType,square_feet,status,floor,furnishing,facing,description,price_per_sqft,price,bhk,property_type,locality,city,transaction_standardized,floor_number,total_floors
0,2 BHK Apartment for Sale in Dindoli Surat,Carpet Area,644,Under Construction,5 out of 10,Unfurnished,West,"Luxury project with basement parking, Solar ro...",2891.0,3380000.0,2.0,Apartment,Dindoli,Surat,New Property,5.0,10.0
1,2 BHK Apartment for Sale in Althan Surat,Super Area,1278,Under Construction,6 out of 14,Unfurnished,South - West,2 And 3 BHK Luxurious Flat for Sell In New Alt...,3551.0,4540000.0,2.0,Apartment,Althan,Surat,New Property,6.0,14.0
2,2 BHK Apartment for Sale in Pal Gam Surat,Super Area,1173,Ready to Move,5 out of 13,Semi-Furnished,East,This affordable 2 BHK flat is situated along a...,3800.0,4460000.0,2.0,Apartment,Pal Gam,Surat,Resale,5.0,13.0
3,2 BHK Apartment for Sale in Jahangirabad Surat,Carpet Area,700,Ready to Move,6 out of 14,Unfurnished,East,2 BHK Flat For sell IN Jahangirabad Prime Loca...,3966.0,4700000.0,2.0,Apartment,Jahangirabad,Surat,New Property,6.0,14.0
4,"2 BHK Apartment for Sale in Orchid Fantasia, P...",Super Area,1250,Ready to Move,Unfurnished,Unfurnished,NaN,"Multistorey Apartment for Sale in Palanpur, Su...",3600.0,4500000.0,2.0,Apartment,"Orchid Fantasia, Palanpur",Surat,New Property,NaN,NaN


## Standardized Furnishing

In [ ]:
print(gdf['furnishing'].unique())

['Unfurnished' 'Semi-Furnished' 'Furnished' nan]


In [ ]:
valid_floor_values = ['Unfurnished', 'Semi-Furnished', 'Furnished']

# Function to map the value from floor to furnishing if furnishing has an invalid value
def map_furnishing_from_floor(row):
    if row['floor'] in valid_floor_values and row['furnishing'] not in valid_floor_values:
        return row['floor']  # Map the value from 'floor' to 'furnishing'
    return row['furnishing']  # Otherwise, keep the original value in 'furnishing'

# Apply the function to the 'furnishing' column
gdf['furnishing'] = gdf.apply(map_furnishing_from_floor, axis=1)

# Display the updated DataFrame
print(gdf)

                                          property_name areaWithType  \
0             2 BHK Apartment for Sale in Dindoli Surat  Carpet Area   
1              2 BHK Apartment for Sale in Althan Surat   Super Area   
2             2 BHK Apartment for Sale in Pal Gam Surat   Super Area   
3        2 BHK Apartment for Sale in Jahangirabad Surat  Carpet Area   
4     2 BHK Apartment for Sale in Orchid Fantasia, P...   Super Area   
...                                                 ...          ...   
4520  6 BHK Apartment for Sale in Millionaires Lifes...  Carpet Area   
4521  4 BHK Apartment for Sale in Savan Superia, Alt...   Super Area   
4522  5 BHK Apartment for Sale in Roongta Green Vall...  Carpet Area   
4523  6 BHK Apartment for Sale in Cellestial Dreams,...  Carpet Area   
4524  4 BHK Apartment for Sale in Roongta Green Vall...   Super Area   

      square_feet              status        floor      furnishing  \
0             644  Under Construction  5 out of 10     Unfurnishe

In [ ]:
furnishing_map = {
    'Unfurnished': 'Unfurnished',
    'Semi-Furnished': 'Semi-Furnished',
    'Furnished': 'Fully Furnished',
    None: None
}

gdf['furnishing_standardized'] = gdf['furnishing'].map(furnishing_map)

In [ ]:

df['furnishing_standardized'] = df['furnishing'].map(furnishing_map)

## Standardized Facing Direcion

In [ ]:
print(gdf['facing'].unique())


['West' 'South - West' 'East' nan 'North - East' 'South - East'
 'North - West' 'South' 'North']


In [ ]:
def standardize_facing_direction(direction):
    if isinstance(direction, str):
        if ' - ' in direction:
            directions = direction.split(' - ')
            return directions[0]
        else:
            return direction  # Keep as it is if no '-'
    return None

gdf['facing_standardized'] = gdf['facing'].apply(standardize_facing_direction)


## Locaion Standardizaion

In [ ]:
gdf.head()

,property_name,areaWithType,square_feet,status,floor,furnishing,facing,description,price_per_sqft,price,bhk,property_type,locality,city,transaction_standardized,floor_number,total_floors,furnishing_standardized,facing_standardized
0,2 BHK Apartment for Sale in Dindoli Surat,Carpet Area,644,Under Construction,5 out of 10,Unfurnished,West,"Luxury project with basement parking, Solar ro...",2891.0,3380000.0,2.0,Apartment,Dindoli,Surat,New Property,5.0,10.0,Unfurnished,West
1,2 BHK Apartment for Sale in Althan Surat,Super Area,1278,Under Construction,6 out of 14,Unfurnished,South - West,2 And 3 BHK Luxurious Flat for Sell In New Alt...,3551.0,4540000.0,2.0,Apartment,Althan,Surat,New Property,6.0,14.0,Unfurnished,South
2,2 BHK Apartment for Sale in Pal Gam Surat,Super Area,1173,Ready to Move,5 out of 13,Semi-Furnished,East,This affordable 2 BHK flat is situated along a...,3800.0,4460000.0,2.0,Apartment,Pal Gam,Surat,Resale,5.0,13.0,Semi-Furnished,East
3,2 BHK Apartment for Sale in Jahangirabad Surat,Carpet Area,700,Ready to Move,6 out of 14,Unfurnished,East,2 BHK Flat For sell IN Jahangirabad Prime Loca...,3966.0,4700000.0,2.0,Apartment,Jahangirabad,Surat,New Property,6.0,14.0,Unfurnished,East
4,"2 BHK Apartment for Sale in Orchid Fantasia, P...",Super Area,1250,Ready to Move,Unfurnished,Unfurnished,NaN,"Multistorey Apartment for Sale in Palanpur, Su...",3600.0,4500000.0,2.0,Apartment,"Orchid Fantasia, Palanpur",Surat,New Property,NaN,NaN,Unfurnished,None


In [ ]:
gdf['locality'].unique()

array(['Dindoli', 'Althan', 'Pal Gam', ..., 'KPM Terra Prime, Piplod',
       'Milestone Palacio, Athwa', 'D and M Elysium Avenue, Piplod'],
      dtype=object)

In [ ]:
gdf['location'] = gdf['locality']

In [ ]:
def extract_locality_city(property_name):
    if pd.isna(property_name):
        return None

    property_name = property_name.replace('for Sale', '').strip()

    if ' in ' in property_name:
        _, location_part = property_name.split(' in ', 1)
    else:
        location_part = property_name

    if ',' in location_part:
        parts = location_part.split(',')
        locality = parts[-1].replace('Surat', '').strip()
    else:
        locality = location_part.replace('Surat', '').strip()

    if locality:
        return f"{locality}, Surat"
    else:
        return None

gdf['location_Standardizaion'] = gdf['property_name'].apply(extract_locality_city)


In [ ]:
gdf.head()

,property_name,areaWithType,square_feet,status,floor,furnishing,facing,description,price_per_sqft,price,...,property_type,locality,city,transaction_standardized,floor_number,total_floors,furnishing_standardized,facing_standardized,location,location_Standardizaion
0,2 BHK Apartment for Sale in Dindoli Surat,Carpet Area,644,Under Construction,5 out of 10,Unfurnished,West,"Luxury project with basement parking, Solar ro...",2891.0,3380000.0,...,Apartment,Dindoli,Surat,New Property,5.0,10.0,Unfurnished,West,Dindoli,"Dindoli, Surat"
1,2 BHK Apartment for Sale in Althan Surat,Super Area,1278,Under Construction,6 out of 14,Unfurnished,South - West,2 And 3 BHK Luxurious Flat for Sell In New Alt...,3551.0,4540000.0,...,Apartment,Althan,Surat,New Property,6.0,14.0,Unfurnished,South,Althan,"Althan, Surat"
2,2 BHK Apartment for Sale in Pal Gam Surat,Super Area,1173,Ready to Move,5 out of 13,Semi-Furnished,East,This affordable 2 BHK flat is situated along a...,3800.0,4460000.0,...,Apartment,Pal Gam,Surat,Resale,5.0,13.0,Semi-Furnished,East,Pal Gam,"Pal Gam, Surat"
3,2 BHK Apartment for Sale in Jahangirabad Surat,Carpet Area,700,Ready to Move,6 out of 14,Unfurnished,East,2 BHK Flat For sell IN Jahangirabad Prime Loca...,3966.0,4700000.0,...,Apartment,Jahangirabad,Surat,New Property,6.0,14.0,Unfurnished,East,Jahangirabad,"Jahangirabad, Surat"
4,"2 BHK Apartment for Sale in Orchid Fantasia, P...",Super Area,1250,Ready to Move,Unfurnished,Unfurnished,NaN,"Multistorey Apartment for Sale in Palanpur, Su...",3600.0,4500000.0,...,Apartment,"Orchid Fantasia, Palanpur",Surat,New Property,NaN,NaN,Unfurnished,None,"Orchid Fantasia, Palanpur","Palanpur, Surat"


## Price per Sqft Category

In [ ]:
# Only fill where price_per_sqft is missing
mask_missing = gdf['price_per_sqft'].isna()

# Calculate price_per_sqft as price divided by square_feet
gdf.loc[mask_missing, 'price_per_sqft'] = gdf.loc[mask_missing, 'price'] / gdf.loc[mask_missing, 'square_feet']


In [ ]:
print(f"Number of price_per_sqft filled: {mask_missing.sum()}")


Number of price_per_sqft filled: 0


In [ ]:
print(gdf['price_per_sqft'].unique())


[2891. 3551. 3800. ... 8512. 8200. 5756.]


In [ ]:
def categorize_price(price):
    if pd.isna(price):
        return None
    if price < 3000:
        return 'Budget'
    elif 3000 <= price <= 6000:
        return 'Mid-Range'
    else:
        return 'Luxury'

gdf['price_category'] = gdf['price_per_sqft'].apply(categorize_price)


In [ ]:
gdf["price_category"].unique()

array(['Budget', 'Mid-Range', 'Luxury'], dtype=object)

# Gold Layer Data Model

In [ ]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4524 entries, 0 to 4524
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   property_name             4524 non-null   object 
 1   areaWithType              4270 non-null   object 
 2   square_feet               4524 non-null   int64  
 3   status                    4524 non-null   object 
 4   floor                     4524 non-null   object 
 5   furnishing                3993 non-null   object 
 6   facing                    2382 non-null   object 
 7   description               3154 non-null   object 
 8   price_per_sqft            4524 non-null   float64
 9   price                     4524 non-null   float64
 10  bhk                       3889 non-null   float64
 11  property_type             3889 non-null   object 
 12  locality                  4524 non-null   object 
 13  city                      4524 non-null   object 
 14  transaction_s

In [ ]:
gdf.drop(columns=['floor' ,'furnishing' , 'locality' , 'facing' ], inplace=True)

In [ ]:
print(gdf.isna().sum())


property_name                  0
areaWithType                 254
square_feet                    0
status                         0
description                 1370
price_per_sqft                 0
price                          0
bhk                          635
property_type                635
city                           0
transaction_standardized      26
floor_number                 770
total_floors                 770
furnishing_standardized      531
facing_standardized         2142
location                       0
location_Standardizaion       48
price_category                 0
dtype: int64


In [ ]:
# Drop rows where price or square_feet are missing if i dant have prive so i dont have the info to sell it
gdf = gdf.dropna(subset=['price', 'square_feet'])


In [ ]:
print(gdf.isna().sum())

property_name                  0
areaWithType                 254
square_feet                    0
status                         0
description                 1370
price_per_sqft                 0
price                          0
bhk                          635
property_type                635
city                           0
transaction_standardized      26
floor_number                 770
total_floors                 770
furnishing_standardized      531
facing_standardized         2142
location                       0
location_Standardizaion       48
price_category                 0
dtype: int64


In [ ]:
df[['description']].head(10)


,description
0,"Luxury project with basement parking, Solar ro..."
1,2 And 3 BHK Luxurious Flat for Sell In New Alt...
2,This affordable 2 BHK flat is situated along a...
3,2 BHK Flat For sell IN Jahangirabad Prime Loca...
4,"Multistorey Apartment for Sale in Palanpur, Su..."
5,Multistorey Apartment for Sale in Jahangirabad...
6,contraction
7,"best commercial for business ,investors ,busin..."
8,"Multistorey Apartment for Sale in Palanpur, Su..."
9,"Irrespective of the business branch, the our g..."


In [ ]:
def safe_string(value):
    return str(value) if isinstance(value, str) else ""

# Function to extract BHK from description
def extract_bhk(description):
    description = safe_string(description)
    match = re.search(r'(\d+)\s*BHK', description)
    if match:
        return match.group(1)
    return None

# Function to extract property type
def extract_property_type(description):
    description = safe_string(description)
    if "Apartment" in description:
        return "Apartment"
    elif "Villa" in description:
        return "Villa"
    elif "Plot" in description:
        return "Plot"
    return None

# Function to extract furnishing type
def extract_furnishing(description):
    description = safe_string(description)
    if "Furnished" in description:
        return "Furnished"
    elif "Semi-Furnished" in description:
        return "Semi-Furnished"
    elif "Unfurnished" in description:
        return "Unfurnished"
    return None

# Function to extract transaction status
def extract_transaction(description):
    description = safe_string(description)
    if "for Sale" in description or "available for sale" in description:
        return "For Sale"
    elif "sold" in description:
        return "Sold"
    elif "rent" in description:
        return "For Rent"
    return None

# Fill NaN values in each column using the appropriate extraction function
gdf['bhk'] = gdf['bhk'].combine_first(gdf['description'].apply(extract_bhk))
gdf['property_type'] = gdf['property_type'].combine_first(gdf['description'].apply(extract_property_type))
gdf['furnishing_standardized'] = gdf['furnishing_standardized'].combine_first(gdf['description'].apply(extract_furnishing))
gdf['transaction_standardized'] = gdf['transaction_standardized'].combine_first(gdf['description'].apply(extract_transaction))

# Check NaN values count after filling
print(gdf.isna().sum())
print(gdf)

property_name                  0
areaWithType                 254
square_feet                    0
status                         0
description                 1370
price_per_sqft                 0
price                          0
bhk                          634
property_type                592
city                           0
transaction_standardized      19
floor_number                 770
total_floors                 770
furnishing_standardized      531
facing_standardized         2142
location                       0
location_Standardizaion       48
price_category                 0
dtype: int64
                                          property_name areaWithType  \
0             2 BHK Apartment for Sale in Dindoli Surat  Carpet Area   
1              2 BHK Apartment for Sale in Althan Surat   Super Area   
2             2 BHK Apartment for Sale in Pal Gam Surat   Super Area   
3        2 BHK Apartment for Sale in Jahangirabad Surat  Carpet Area   
4     2 BHK Apartment for Sale in

In [ ]:
len(gdf['property_name'].unique())


1991

## property_id

In [ ]:
gdf['property_id'] = [str(uuid.uuid4()) for _ in range(len(gdf))]


## property_name

In [ ]:
gdf["property_name"].unique()

array(['2 BHK Apartment for Sale in Dindoli Surat',
       '2 BHK Apartment for Sale in Althan Surat',
       '2 BHK Apartment for Sale in Pal Gam Surat', ...,
       '5 BHK Apartment for Sale in Sangini Arise, Bharthana Surat',
       '5 BHK Apartment for Sale in D and M Elysium Avenue, Piplod Surat',
       '6 BHK Apartment for Sale in Millionaires Lifestyle, Vesu Surat'],
      dtype=object)

In [ ]:
def standardize_property_name(name):
    if pd.isna(name):
        return name  # leave NaNs as is
    name = name.strip()            # Remove leading/trailing spaces
    name = ' '.join(name.split())  # Remove extra spaces between words
    name = name.title()            # Make Title Case (First Letter Capital)
    return name

gdf['property_name'] = gdf['property_name'].apply(standardize_property_name)


## locaion

In [ ]:
def extract_location(property_name):
    if pd.isna(property_name):
        return None

    match = re.search(r'in (.+)', property_name, re.IGNORECASE)
    if match:
        location_part = match.group(1).strip()
        return location_part
    else:
        return None

gdf['location'] = gdf['property_name'].apply(extract_location)


In [ ]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4524 entries, 0 to 4524
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   property_name             4524 non-null   object 
 1   areaWithType              4270 non-null   object 
 2   square_feet               4524 non-null   int64  
 3   status                    4524 non-null   object 
 4   description               3154 non-null   object 
 5   price_per_sqft            4524 non-null   float64
 6   price                     4524 non-null   float64
 7   bhk                       3890 non-null   object 
 8   property_type             3932 non-null   object 
 9   city                      4524 non-null   object 
 10  transaction_standardized  4505 non-null   object 
 11  floor_number              3754 non-null   float64
 12  total_floors              3754 non-null   float64
 13  furnishing_standardized   3993 non-null   object 
 14  facing_standa

## square_feet

In [ ]:
gdf['square_feet'] = gdf['square_feet'].astype('Int64')

## areaWithType

In [ ]:
gdf['areaWithType'].unique()

array(['Carpet Area', 'Super Area', nan], dtype=object)

## transaction_type

In [ ]:
gdf['transaction'] = gdf['transaction_standardized']

In [ ]:
gdf['transaction'].unique()

array(['New Property', 'Resale', 'For Sale', None], dtype=object)

## status

In [ ]:
gdf['status'].unique()

array(['Under Construction', 'Ready to Move', '1 out of 7', '21 X 50.2',
       '3 out of 10', '120 X 105', '3 out of 7', '36 X 15', '2',
       '3 out of 3', '52.5 X 32.1', '3 out of 5', '5 out of 5',
       '12 X 35.92', '13 out of 14', '7.29 X 14', '4 out of 5',
       'Ground out of 1', '3 out of 4', 'Ground out of 10',
       'Ground out of 2', '20 X 40', '16.4 X 60', '11 out of 14',
       '9.42 X 126', '25 X 12', '2 out of 4', '54 X 15', '21 X 57',
       '26.7 X 55.6', '41 X 20', '5 out of 13', '36 X 21', '19.2 X 54.8',
       '8 out of 14', '26.7 X 54.9', '38 X 21', '2 out of 13', '135 X 75',
       '38 X 20', '114 X 60', '45 X 40', '40 X 21', '37 X 26',
       '180 X 22.5', '4 out of 14', '114 X 14', '285 X 51', '8 out of 10',
       '6 out of 11', 'Ground out of 5', '10 out of 11', '5 out of 14',
       '60 X 28', '37 X 20', '13.3 X 135', '9 out of 11', '48 X 14',
       '45.5 X 21.56', '60 X 120', '8 out of 13', '7 out of 13',
       '4.55 X 20', '9 out of 13', '16.7 X 240'

In [ ]:
valid_statuses = ['Under Construction', 'Ready to Move']

# Update 'status' column
gdf['status'] = gdf['status'].apply(lambda x: x if x in valid_statuses else np.nan)

# Check the unique values in the 'status' column
print(gdf['status'].unique())

['Under Construction' 'Ready to Move' nan]


## floor_number  

In [ ]:
gdf['floor_number'].unique()

array([ 5.,  6., nan,  7.,  3.,  1.,  4.,  2.,  0.,  9., 12., 10., 13.,
        8., 14., 16., 11., 15., 18., 20., 17., 19.])

## total_floors

In [ ]:
gdf['total_floors'].unique()

array([10., 14., 13., nan,  1.,  7.,  3.,  6.,  5., 12.,  2.,  4.,  9.,
        8., 11.,  0., 19., 15., 30., 20., 21., 18., 16., 17., 25.])

## furnishing

In [ ]:
gdf['furnishing'] = gdf['furnishing_standardized']

In [ ]:
gdf['furnishing'].unique()

array(['Unfurnished', 'Semi-Furnished', 'Fully Furnished', None],
      dtype=object)

## facing

In [ ]:
gdf['facing'] = gdf['facing_standardized']

In [ ]:
gdf['facing'].unique()

array(['West', 'South', 'East', None, 'North'], dtype=object)

## price_per_sqft

In [ ]:
gdf['price_per_sqft'].unique()

array([2891., 3551., 3800., ..., 8512., 8200., 5756.])

## price

In [ ]:
gdf['price'].unique()

array([ 3380000.,  4540000.,  4460000.,  4700000.,  4500000.,  4320000.,
        4210000.,  4410000.,  4430000.,  4000000.,  4400000.,  1250000.,
        5000000.,  3580000.,  2650000.,  4200000.,  4300000.,  2000000.,
        3700000.,  2900000.,  3800000.,  1300000.,  4650000.,  4160000.,
        3350000.,  4250000.,  4800000.,  3200000.,  3500000.,  4620000.,
        3850000.,  2300000.,  2400000.,  2800000.,  3570000.,  3650000.,
        3600000.,  4360000.,  2500000.,  4100000.,  2850000.,   900000.,
        3000000.,  3950000.,  2150000.,  1050000.,  1800000.,  1000000.,
        2350000.,  1630000.,  1350000.,  4900000.,  2100000.,  3900000.,
        1600000.,  2700000.,  1500000.,  3990000.,   750000.,  1280000.,
         950000.,  3150000.,  4600000.,   600000.,  3400000.,  2200000.,
        1900000.,   850000.,  1100000.,  1650000.,  1700000.,  1400000.,
        3300000.,  2050000.,  2170000.,  2750000.,  1200000.,  2600000.,
         770000.,  4480000.,  3490000.,  1950000., 

## price_category

In [ ]:
gdf['price_category'].unique()

array(['Budget', 'Mid-Range', 'Luxury'], dtype=object)

In [ ]:
gdf.drop(columns=['location_Standardizaion' , 'city' , 'furnishing', 'description'], inplace=True ,axis= 1)

In [ ]:
gdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4524 entries, 0 to 4524
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   property_name             4524 non-null   object 
 1   areaWithType              4270 non-null   object 
 2   square_feet               4524 non-null   Int64  
 3   status                    4400 non-null   object 
 4   price_per_sqft            4524 non-null   float64
 5   price                     4524 non-null   float64
 6   bhk                       3890 non-null   object 
 7   property_type             3932 non-null   object 
 8   transaction_standardized  4505 non-null   object 
 9   floor_number              3754 non-null   float64
 10  total_floors              3754 non-null   float64
 11  furnishing_standardized   3993 non-null   object 
 12  facing_standardized       2382 non-null   object 
 13  location                  4524 non-null   object 
 14  price_categor